# Get Top 100 Billboard Charts for entered date

In [113]:
date = input("Which year do you want to travel to? Type the date in this format YYYY-MM-DD: ")
date

'1990-06-08'

In [126]:
import requests
from bs4 import BeautifulSoup

context = requests.get(f"https://www.billboard.com/charts/hot-100/{date}/").text
soup = BeautifulSoup(context, "html.parser", from_encoding="utf-8")

/Users/Lukas.Deibel/code_repository/100_days_of_code/lib/python3.12/site-packages/bs4/__init__.py:228: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


<span class="c-label  a-font-primary-bold-l u-font-size-32@tablet u-letter-spacing-0080@tablet">

<h3 id="title-of-a-story" class="c-title  a-no-trucate a-font-primary-bold-s u-letter-spacing-0021 lrv-u-font-size-18@tablet lrv-u-font-size-16 u-line-height-125 u-line-height-normal@mobile-max a-truncate-ellipsis u-max-width-330 u-max-width-230@tablet-only">

	
	
		
					Everything You Want		
	
</h3>

h3 class="c-title a-no-trucate a-font-primary-bold-s u-letter-spacing-0021 lrv-u-font-size-18@tablet lrv-u-font-size-16 u-line-height-125 u-line-height-normal@mobile-max a-truncate-ellipsis u-max-width-330 u-max-width-230@tablet-only" id="title-of-a-story"

In [115]:
song_names_spans = soup.select("div ul li ul li h3")
song_names = [song_name.get_text(strip=True) for song_name in song_names_spans]

In [116]:
song_names[:5]

['Hold On',
 'Vogue',
 'Poison',
 'All I Wanna Do Is Make Love To You',
 'It Must Have Been Love (From "Pretty Woman")']

In [117]:
artist_names_spans = soup.select("li ul li span")
artist_names = [artist_name.get_text(strip=True) for artist_name in artist_names_spans][::7]

In [118]:
artist_names[:5]

['Wilson Phillips', 'Madonna', 'Bell Biv DeVoe', 'Heart', 'Roxette']

# Authenticate to Spotify

In [119]:
import os
client_id = os.environ.get("SPOTIPY_CLIENT_ID")
client_secret = os.environ.get("SPOTIPY_CLIENT_SECRET")
client_id, client_secret

('2ec2093386bd4d9fabc19f48828f613b', '1358e693e3ae40fe805ed4577dd767be')

In [125]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

sp = spotipy.Spotify(
    auth_manager=SpotifyOAuth(
        scope="playlist-modify-private",
        redirect_uri="http://example.com",
        client_id=os.environ.get("SPOTIPY_CLIENT_ID"),
        client_secret=os.environ.get("SPOTIPY_CLIENT_SECRET"),
        show_dialog=True,
        cache_path="./token.txt",
        username="lueksche", 
    )
)

In [121]:
current_user = sp.current_user().get("id")

In [122]:
sp.user_playlist_create(user=current_user, name=f"Billboard Hot 100 - {date}", public=True)

{'collaborative': False,
 'description': '',
 'external_urls': {'spotify': 'https://open.spotify.com/playlist/4NEAffVOZUYrKW2yk0gE6Y'},
 'followers': {'href': None, 'total': 0},
 'href': 'https://api.spotify.com/v1/playlists/4NEAffVOZUYrKW2yk0gE6Y',
 'id': '4NEAffVOZUYrKW2yk0gE6Y',
 'images': [],
 'primary_color': None,
 'name': 'Billboard Hot 100 - 1990-06-08',
 'type': 'playlist',
 'uri': 'spotify:playlist:4NEAffVOZUYrKW2yk0gE6Y',
 'owner': {'href': 'https://api.spotify.com/v1/users/diabl00d',
  'id': 'diabl00d',
  'type': 'user',
  'uri': 'spotify:user:diabl00d',
  'display_name': None,
  'external_urls': {'spotify': 'https://open.spotify.com/user/diabl00d'}},
 'public': True,
 'snapshot_id': 'AAADP9arT8pFDjLqEMR0mbacSXLGskts',
 'tracks': {'limit': 100,
  'next': None,
  'offset': 0,
  'previous': None,
  'href': 'https://api.spotify.com/v1/playlists/4NEAffVOZUYrKW2yk0gE6Y/tracks',
  'total': 0,
  'items': []}}

In [123]:
playlist_id = sp.current_user_playlists().get("items")[0].get("uri")
playlist_id

'spotify:playlist:4NEAffVOZUYrKW2yk0gE6Y'

In [124]:
for song_name, artist_name in zip(song_names, artist_names):
    results = sp.search(q=f"track:{song_name} artist:{artist_name}", type="track")
    if results.get("tracks").get("items"):
        track_id = results.get("tracks").get("items")[0].get("id")
        sp.playlist_add_items(playlist_id, items=[track_id])